# **ORION - Component Analysis and Inference Demo**

In this notebook, we will analyze the functions implemented for each of the components in ORION's architecture to analyze how the data flow is processed in each stage of the vision, reasoning and action space, during the inference process and we will also take a look at a demo to test the ORION inference pipeline.

#**Installing Dependencies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/RA-KaiKaiLiu/Orion')

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda --version

conda 24.11.2


In [ ]:
%%bash
conda update -n base -c conda-forge conda -y
conda create -n orion_env python=3.9 -y

apt-get update
apt-get install -y cuda-toolkit-11-8 libcudnn8-dev build-essential ninja-build

if [ -d /usr/local/cuda ]; then sudo rm -rf /usr/local/cuda; fi
sudo ln -s /usr/local/cuda-11.8 /usr/local/cuda



Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/orion_env

  added / updated specs:
    - python=3.9


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-hda65f42_8 
  ca-certificates    conda-forge/noarch::ca-certificates-2025.11.12-hbd8a1cb_0 
  icu                conda-forge/linux-64::icu-75.1-he02047a_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.45-h1aa0949_0 
  libexpat           conda-forge/linux-64::libexpat-2.7.3-hecca717_0 
  libffi             conda-forge/linux-64::libffi-3.5.2-h9ec8514_0 
  libgcc             conda-forge/linux-64::libgcc-15.2.



==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda




==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
%%bash
conda run -n orion_env /bin/bash -c '

pip install "pip<25.3" "setuptools<64"

export CUDA_HOME=/usr/local/cuda
export PATH=/usr/local/cuda/bin:$PATH
export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
export CFLAGS="-I/usr/local/cuda/include"
export LDFLAGS="-L/usr/local/cuda/lib64"


pip install -r /content/drive/MyDrive/RA-KaiKaiLiu/Orion/requirements.txt

conda install -c conda-forge numba llvmlite

pip install torch==2.4.1+cu118 torchvision==0.19.1+cu118 torchaudio==2.4.1 --extra-index-url https://download.pytorch.org/whl/cu118

pip install flash-attn==2.6.2 --no-build-isolation

pip install ninja

pip install google-generativeai pillow

pip install -v -e .

'

Ignoring regex: markers 'sys_platform == "win32"' don't match your environment
Ignoring pycocotools-windows: markers 'platform_system == "Windows"' don't match your environment
Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached flash_attn-2.6.2.tar.gz (2.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for flash-attn: filename=flash_attn-2.6.2-cp39-cp39-linux_x86_64.whl size=189313476 sha256=498f96a455b597514db9057047e0ebccfa1cb789d60cdc3e4859221167fbdae8
  Stored in directory: /root/.cache/pip/wheels/8a/a8/bf/10928691af7693c48404a649f0daa479c94df0064ed1771a26
Successfully built flash-attn
Using pip 25.2 from /usr/local/envs/orion_env/lib/python3.9/site-packages/pip (python 3.9)
Obtaining file:///content/drive/MyDrive/RA-KaiKaiLiu/Orion




==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda


  DEPRECATION: Building 'flash-attn' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'flash-attn'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-nfo6binq/mmcv.egg-info
  writing /tmp/pip-pip-egg-info-nfo6binq/mmcv.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-nfo6binq/mmcv.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-nfo6binq/mmcv.egg-i

#**Inference steps**



The complete inference flow is:

* [test.py](https://github.com/xiaomi-mlab/Orion/blob/main/adzoo/orion/test.py) → loads config and checkpoint
* forward_test() → resets memory, calls simple_test_pts()
* extract_feat() → backbone + neck feature extraction
* Detection heads → pts_bbox_head.forward() and map_head.forward()
* VLM → lm_head.inference_ego() generates planning token
* Planning decoder → VAE/Diffusion/MLP generates trajectory
* Metrics → compute planning metrics (L2 error, collision, etc.)

During inference, the following methods in [orion.py](https://github.com/xiaomi-mlab/Orion/blob/2eddb627/mmcv/models/detectors/orion.py) are called in sequence:
<ul type="none">
<li>a) forward_test() - Entry point for test-time inference</li>
 <ul>
<li>Resets memory if needed </li>
<li>Delegates to simple_test() </li>
</ul>
<li>b) simple_test() - Processes single scene</li>
<ul>
<li>Calls extract_img_feat() to extract image features</li>
<li>Calls simple_test_pts() for the main inference pipeline</li>
</ul>
<li>c) simple_test_pts() - Core inference pipeline</li>
<li>This is where the three-stage processing happens:</li>
<ul>
<li>Vision Stage: Calls pts_bbox_head() for object detection and map_head() for lane detection</li>
<li>Reasoning Stage: Calls lm_head.inference_ego() or lm_head.generate() for VLM processing</li>
<li>Action Stage: Depending on config, calls trajectory decoders:
<ul><li>VAE decoder: ego_fut_decoder()</li><li>Diffusion decoder: diff_decoder()</li><li>MLP decoder: waypoint_decoder()</li></ul></li>
</ul>

#**Vision Space Implementation Overview**
##Image Loading & Preprocessing
Pipeline: inference_only_pipeline in agent configuration [orion_stage3_agent.py](https://github.com/Rahhul17-IITH/Orion-v1/blob/main/adzoo/orion/configs/orion_stage3_agent.py)

**Input:**

* Raw images from CARLA: 6 camera views (RGB, variable resolution) [orion_b2d_agent.py](https://github.com/Rahhul17-IITH/Orion-v1/blob/main/team_code/orion_b2d_agent.py)

**Processing:**

* LoadMultiViewImageFromFilesInCeph: Load images to float32 [orion_stage3_agent.py](https://github.com/Rahhul17-IITH/Orion-v1/blob/main/adzoo/orion/configs/orion_stage3_agent.py)
* ResizeCropFlipRotImage: Apply image transforms (no augmentation during inference)
* ResizeMultiview3D: Resize to (640, 640)
* NormalizeMultiviewImage: Apply normalization
* PadMultiViewImage: Pad to divisible by 32

**Output:**

* results['img']: List of 6 images, stacked to (6, 3, 640, 640) tensor [orion_b2d_agent.py](https://github.com/Rahhul17-IITH/Orion-v1/blob/main/team_code/orion_b2d_agent.py)

## Feature Extraction
**Function:** extract_img_feat() called from simple_test() [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* img: (B, N, C, H, W) = (1, 6, 3, 640, 640) tensor (dtype: float32 or float16)

**Processing:**

* Reshape: (B*N, C, H, W) = (6, 3, 640, 640)
* img_backbone.forward() (EVAViT): Extract features
* Reshape back: (B, N, C_feat, H_feat, W_feat)

**Output:**

* img_feats_reshaped: (1, 6, 1024, 40, 40) tensor - 1024-dim features at 40×40 resolution

## Position Encoding
**Function:** position_embeding() in simple_test_pts() [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* img_feats: (1, 6, 1024, 40, 40)
* location: (6, 40, 40, 2) - 2D grid coordinates from prepare_location()

**Processing:**

* Flatten spatial: (B, N*H*W, C) = (1, 9600, 1024)
* Generate depth bins and encode via position_encoder MLP

**Output:**

* pos_embed: (1, 9600, 256) - positional embeddings

## Object Detection
* Function: pts_bbox_head.forward() (OrionHead) [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* img_feats: (1, 6, 1024, 40, 40)
* pos_embed: (1, 9600, 256)
* img_metas: List with camera calibration, ego pose, etc.

**Processing:**

* Flatten & project: memory = input_projection(img_feats) → (1, 9600, 256)
* Initialize 600 object queries + 256 VLM tokens
* 6-layer transformer decoder with temporal memory
* Extract VLM tokens from last layer: (1, 256, 256)
* Project to 4096-dim: (1, 256, 4096)
* Add CAN bus embedding: (1, 257, 4096) (256 object + 1 planning token)

**Output:**

* outs_bbox: Dict with detection predictions (bboxes, classes, scores)
* det_query: (1, 257, 4096) tensor - vision tokens for VLM

## Map Detection
* Function: map_head.forward() (OrionHeadM) [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* img_feats: (1, 6, 1024, 40, 40)
* pos_embed: (1, 9600, 256)

**Processing:**

* Similar to OrionHead but with 1800 lane queries + 256 VLM tokens
* 6-layer transformer decoder
* Extract & project VLM tokens: (1, 256, 4096)

**Output:**

* outs_lane: Dict with lane predictions (coordinates, classes)
* map_query: (1, 256, 4096) tensor - map vision tokens

# Bounding Box Results [orion.py](https://github.com/xiaomi-mlab/Orion/blob/2eddb627/mmcv/models/detectors/orion.py)
This file is saved as an array of dictionaries, where each dictionary contains the detection results for a single sample or time step. The common keys and their meanings are:

* boxes_3d: 3D bounding boxes (LiDARInstance3DBoxes format)
* scores_3d: Confidence scores for each detection
* labels_3d: Class labels (0-8 for the 9 detection classes)
* trajs_3d (optional): Future trajectory predictions if motion forecasting is enabled

# Lane Results [orion.py](https://github.com/xiaomi-mlab/Orion/blob/2eddb627/mmcv/models/detectors/orion.py)
* map_scores_3d: Confidence scores for detected lanes
* map_labels_3d: Lane class labels (6 classes: Broken, Solid, SolidSolid, Center, TrafficLight, StopSign)
* map_pts_3d: Lane control points (shape: [num_lanes, n_control, 3])

#**Reasoning Space Implementation Overview**
## Vision Token Concatenation
* Function: In simple_test_pts() [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* vision_embeded_obj: (1, 257, 4096) from OrionHead
* vision_embeded_map: (1, 256, 4096) from OrionHeadM

**Processing:**

* vision_embeded = torch.cat([vision_embeded_obj, vision_embeded_map], dim=1) [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Output:**

* vision_embeded: (1, 513, 4096) tensor (dtype: float32 or float16)
## Text Input Preparation
* Pipeline: LoadAnnoatationCriticalVQATest transform [orion_stage3_agent.py](https://github.com/Rahhul17-IITH/Orion-v1/blob/main/adzoo/orion/configs/orion_stage3_agent.py)

**Input:**

* Critical QA prompts from dataset (e.g., "Describe the driving scenario and plan waypoints")

**Processing:**

* Tokenize using LLaVA tokenizer
* Format as conversation with special tokens

**Output:**

* input_ids: List of token sequences, each (seq_len,) tensor [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

## Multi-Turn Conversation Loop
* Function: Loop in simple_test_pts() [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

For each conversation turn:

**Input:**

* input_ids[i]: (1, turn_len) - current turn tokens orion.py:769
* history_input_output_id: List of previous turns [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Processing:**

* Check for special waypoint token (<waypoint_ego>) [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)
* Concatenate conversation history: context_input_ids = torch.cat(history_input_output_id, dim=-1)

**Output:**

* context_input_ids: (1, total_seq_len) - full conversation context
## LLM Inference for Planning Token
* Function: lm_head.inference_ego() [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* inputs: (1, total_seq_len) - conversation tokens [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)
* images: (1, 513, 4096) - vision embeddings
* return_ego_feature=True

**Processing:** (inside LlavaLlamaForCausalLM.inference_ego())

* Merge vision and text: prepare_inputs_labels_for_multimodal() creates inputs_embeds of shape (1, 513+total_seq_len, 4096) [llava_llama.py](https://github.com/xiaomi-mlab/Orion/blob/2eddb627/mmcv/utils/llava_llama.py)
* LLaMA forward pass: self.model(inputs_embeds=inputs_embeds) → hidden_states of shape (1, 513+total_seq_len, 4096)
* Extract planning token: Find <waypoint_ego> token position and extract its hidden state

**Output:**

* ego_feature: (1, 4096) tensor (dtype: float32) - planning token [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)
## LLM Inference for Text Generation (Non-Planning Turns)
* Function: lm_head.generate()

**Input:**

* inputs: (1, total_seq_len) - conversation tokens
* images: (1, 513, 4096) - vision embeddings
* Generation params: temperature=0.1, top_p=0.75, max_new_tokens=320

**Processing:**

* Similar vision-text merging
* Autoregressive generation with sampling

**Output:**

* output_ids: (1, generated_len) tensor - generated token IDs [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

#**Action Space Implementation Overview**
## Planning Token Preparation
**Function:** In simple_test_pts() [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* ego_feature: (1, 4096) tensor

**Processing:**

* Convert to float32: ego_feature = ego_feature.to(torch.float32) [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)
* Add time dimension: current_states = ego_feature.unsqueeze(1) → (1, 1, 4096)

**Output:**

* current_states: (1, 1, 4096) tensor

# Trajectory Generation (VAE Mode)
* Function: VAE-based planning in simple_test_pts() [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)

**Input:**

* current_states: (1, 1, 4096)

**Processing:**

* Distribution sampling: distribution_forward(current_states, None, None) → sample of shape (1, 32) (latent code) [orion.py](https://github.com/xiaomi-mlab/Orion/blob/main/mmcv/models/detectors/orion.py)
* Future state prediction: future_states_predict(B, sample, hidden_states, current_states) → states_hs of shape (6, 1, 1, 4096) (6 timesteps)
* Trajectory decoding: For each timestep, ego_fut_decoder(ego_query_hs[i]) → (1, 6, 2) (6 modes, 2D coords)
* Stack timesteps: ego_fut_preds = torch.stack(ego_fut_trajs_list, dim=2) → (1, 6, 6, 2)

#**Inference Pipeline Demo**

I have created a test file for testing the inference pipeline by loading loading the checkpoint and calling the respective functions implemented for the components in ORION's architecture. [link text](https://)

In [ ]:
%%bash
# Re-install flash-attn in the orion_env, ensuring CUDA environment variables are set.
conda run -n orion_env /bin/bash -c '
export CUDA_HOME=/usr/local/cuda
export PATH=/usr/local/cuda/bin:$PATH
export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
export CFLAGS="-I/usr/local/cuda/include"
export LDFLAGS="-L/usr/local/cuda/lib64"
pip install flash-attn==2.6.2 --no-build-isolation
'
# Then, run the demo script
conda run -n orion_env python notebooks/Exp_FunctionsInspection/Inference_pipeline_demo.py


======== shape of rope freq torch.Size([256, 64]) ========
======== shape of rope freq torch.Size([1600, 64]) ========
load checkpoint from local path: ckpts/Orion.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: lm_head.base_model.model.model.layers.0.self_attn.rotary_emb.cos_cached, lm_head.base_model.model.model.layers.0.self_attn.rotary_emb.sin_cached, lm_head.base_model.model.model.layers.1.self_attn.rotary_emb.cos_cached, lm_head.base_model.model.model.layers.1.self_attn.rotary_emb.sin_cached, lm_head.base_model.model.model.layers.2.self_attn.rotary_emb.cos_cached, lm_head.base_model.model.model.layers.2.self_attn.rotary_emb.sin_cached, lm_head.base_model.model.model.layers.3.self_attn.rotary_emb.cos_cached, lm_head.base_model.model.model.layers.3.self_attn.rotary_emb.sin_cached, lm_head.base_model.model.model.layers.4.self_attn.rotary_emb.cos_cached, lm_head.base_model.model.model.layers.4.self_attn.rotary_emb.sin_cached, lm_head.ba

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/envs/orion_env/lib/python3.9/site-packages/transformers/modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start 